# Day 24
https://adventofcode.com/2016/day/24

In [1]:
import aocd
data = aocd.get_data(year=2016, day=24)

In [2]:
from collections import deque
from dataclasses import dataclass
from itertools import combinations, permutations

In [3]:
@dataclass(frozen=True)
class Point():
    x: int
    y: int
    
    def __add__(self, other):
        return Point(self.x+other.x, self.y+other.y)

In [4]:
class Map():
    def __init__(self, text):
        rows = text.split('\n')
        self.y_size = len(rows)
        self.x_size = len(rows[0])
        self.walls = set()
        self.poi = dict()
        for y, row in enumerate(rows):
            for x, char in enumerate(row):
                if char == '#':
                    self.walls.add(Point(x, y))
                elif char != '.':
                    self.poi[int(char)] = Point(x, y)
    
    def all_possible_moves(self, location):
        for direction in (Point(1, 0), Point(-1, 0), Point(0, 1), Point(0, -1)):
            new_location = location + direction
            if (0 <= new_location.x <= self.x_size
                and 0 <= new_location.y <= self.y_size
                and new_location not in self.walls):
                yield new_location
    
    def shortest_path(self, start, finish):
        visited = set()
        search = deque([[start]])
        
        while search:
            route = search.popleft()
            location = route[-1]
            if location == finish:
                return len(route) - 1
            if location not in visited:
                visited.add(location)
                search.extend(route + [newloc] for newloc in self.all_possible_moves(location))
    
    def poi_routes(self):
        routes = dict()
        for (first, second) in combinations(self.poi.keys(), 2):
            dist = self.shortest_path(self.poi[first], self.poi[second])
            routes[(first, second)] = dist
            routes[(second, first)] = dist
        return routes

In [5]:
def journey_length(poi_routes, journey):
    return sum(poi_routes[(journey[x], journey[x+1])] for x in range(len(journey)-1))

In [6]:
def shortest_journey(maze, comeback=False):
    poi_routes = maze.poi_routes()
    journeys = [
        [0] + list(perm) + ([0] if comeback else [])
        for perm in permutations(poi for poi in maze.poi.keys() if poi != 0)
    ]
    return min(journey_length(poi_routes, journey) for journey in journeys)

In [7]:
maze = Map(data)
p1 = shortest_journey(maze)
print('Part 1: {}'.format(p1))
p2 = shortest_journey(maze, comeback=True)
print('Part 2: {}'.format(p2))

Part 1: 428
Part 2: 680
